Useful links:
* https://gwcs.readthedocs.io/en/latest/index.html
* https://docs.astropy.org/en/stable/modeling/new-model.html#defining-new-model-classes (

In [2]:
import astropy.units as u
import numpy as np
from astropy.modeling import Model, Parameter
from gwcs import coordinate_frames as cf

from overlappy.wcs import overlappogram_fits_wcs, pcij_matrix

In [2]:
class P2W(Model):
    n_inputs = 4
    n_outputs = 4
    alpha = Parameter()
    gamma = Parameter()
    
    @staticmethod
    def order_to_index_mapping(index):
        return [-3, -1, 0, +1, +3][index]
    
    @staticmethod
    def evaluate(p1, p2, p3, p4, alpha, gamma):
        m = P2W.order_to_index_mapping(p4)
        x = np.cos(alpha) * p1 + np.sin(alpha) * p2 - np.sin(alpha - gamma) * p3 * p4
        y = -np.sin(alpha) * p1 + np.cos(alpha) * p2 - np.cos(alpha - gamma) * p3 * p4
        l = p3
        m = p4
        return x, y, l, m
    
    @property
    def inverse(self):
        return W2P(alpha=self.alpha, gamma=self.gamma)

In [3]:
class W2P(Model):
    n_inputs = 4
    n_outputs = 4
    alpha = Parameter()
    gamma = Parameter()
    
    @staticmethod
    def evaluate(x, y, l, m, alpha, gamma):
        A = l * m
        sa = np.sin(alpha)
        ca = np.cos(alpha)
        sg = np.sin(gamma)
        cg = np.cos(gamma)
        sag = np.sin(alpha - gamma)
        cag = np.cos(alpha - gamma)
        p2 = (y + A * cag + (sa / ca) * (x + A * sag)) / ((sa**2 / ca) - sa)
        p1 = (x * A * sag - sa * p2) / ca
        p3 = l
        p4 = m
        return p1, p2, p3, p4
    
    @property
    def inverse(self):
        return P2W(alpha=self.alpha, gamma=self.gamma)

In [82]:
a = P2W(alpha=0.5, gamma=0.1)

In [83]:
a(1, 2, 3, 0)

(1.8364336390987783, 1.275739585176542, 3.0, 0.0)

In [84]:
a.inverse(1.8364336390987783, 1.275739585176542, 3.0, 0.0)

(5.723841904984076, -10.477422328096562, 3.0, 0.0)

In [ ]:
pipeline = [(detector_frame = cf.Frame2D(name="detector", axes_names=("x", "y"),

                            unit=(u.pix, u.pix)))]

In [4]:
cf.CompositeFrame?

In [10]:
mode_frame = cf.CoordinateFrame(1, "SPECTRAL", (3,), name="spectral order")

In [13]:
mode_frame

<CoordinateFrame(name="spectral order", unit=(Unit(dimensionless),), axes_names=('',), axes_order=(3,))>

In [11]:
cf.SpectralFrame()

<CoordinateFrame(name="spectral order", unit=(Unit(dimensionless),), axes_names=('',), axes_order=(3,))>

In [12]:
cf.SpectralFrame?

In [3]:
from sunpy.coordinates import get_earth

In [8]:
det_shape = [475,1073]#*u.pix
pc_matrix = pcij_matrix(0*u.deg, 0*u.deg, order=1)
obs = get_earth()
overlap_wcs = overlappogram_fits_wcs(det_shape,
                       np.arange(1, 60, 0.05)*u.angstrom,
                       (5*u.arcsec/u.pix,5*u.arcsec/u.pix,0.05*u.angstrom/u.pix),
                       pc_matrix=pc_matrix,
                       observer=obs
                      )

In [13]:
overlap_wcs.to_header()

WCSAXES =                    3 / Number of coordinate axes                      
CRPIX1  =                537.0 / Pixel coordinate of reference point            
CRPIX2  =                238.0 / Pixel coordinate of reference point            
CRPIX3  =                  1.0 / Pixel coordinate of reference point            
PC2_3   =                 -1.0 / Coordinate transformation matrix element       
CDELT1  =   0.0013888888888889 / [deg] Coordinate increment at reference point  
CDELT2  =   0.0013888888888889 / [deg] Coordinate increment at reference point  
CDELT3  =                5E-12 / [m] Coordinate increment at reference point    
CUNIT1  = 'deg'                / Units of coordinate increment and value        
CUNIT2  = 'deg'                / Units of coordinate increment and value        
CUNIT3  = 'm'                  / Units of coordinate increment and value        
CTYPE1  = 'HPLN-TAN'           / Coordinate type codegnomonic projection        
CTYPE2  = 'HPLT-TAN'        